In [3]:
import LinearAlgebra as la
using BenchmarkTools: @btime
include("hoSystem.jl");

In [6]:
l_0 = 10   # number of basis functions
ω = 0.25  # strength of harmonic oscillator potential
a = 0.25  # shielding term in coloumb interaction
grid = [x for x in range(-10, stop = 10, length = 2001)]
n = 2     # number of particles

@time system = System(n, HOBasis(l_0, ω, a), grid);

  0.107007 seconds (1.66 k allocations: 7.145 MiB)


In [ ]:
@time inner_ints(system.spfs, system.grid, system.basis.a);

In [ ]:
function getP(C, n)
    l = size(C)[1]
    P = ComplexF64.(la.zeros((l, l)))
    
    for a in 1:l
        for b in 1:l
            for i in 1:n
                P[b, a] += conj(C[a, i]) * C[b, i]
            end
        end
    end
    return P
end

function getF(C, h, u, n)
    l = size(C)[1]
    F = ComplexF64.(la.zeros((l, l)))
    P = getP(C, n)
    
    F .+= h
    for a in 1:l
        for b in 1:l
            for c in 1:l
                for d in 1:l
                    F[a, b] += P[d, c] * u[a, c, b, d]
                end
            end
        end
    end
    return F
end

In [ ]:
function SCF(C, system, iters)
    for i in 1:iters
        F = getF(C, system.h, system.u, system.n)
        C = la.eigvecs(F)
    end
    return C
end

In [ ]:
C_initial = ComplexF64.(la.I(system.l))

C = SCF(C_initial, system, 20);
#C_GHF = SCF(C_initial, system, 20000)